In [8]:
pip install python-Levenshtein

Defaulting to user installation because normal site-packages is not writeable
     ------------------------------------ 100.0/100.0 kB 191.5 kB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 1.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
from fuzzywuzzy import fuzz
import jieba
import os

In [2]:
# Define root directory
root_dir = r"D:/铁书宁/2025 HKU predoc/RA task1_20250405"

df_path = os.path.join(root_dir, "ar-sale-buyer-match_forShuning.csv")
df = pd.read_csv(df_path)

### Step 1: City match for the AR buyer and customer.

In [10]:
# Import region dataset
city_path = os.path.join(root_dir, "China-City-List-latest.csv")
city_df = pd.read_csv(city_path, encoding = 'utf-8', header = 1)

# Keep city names
city_df = city_df[['Adm1_Name_ZH','Adm2_Name_ZH']]
city_df = city_df.drop_duplicates()

# Delele the word "省" and "市"
city_df['Adm1_Name_ZH'] = city_df['Adm1_Name_ZH'].str[:-1]
city_df['Adm2_Name_ZH'] = city_df['Adm2_Name_ZH'].str[:-1]

# Generate a region list
regions = pd.concat([city_df['Adm1_Name_ZH'], city_df['Adm2_Name_ZH']])
regions = list(regions.drop_duplicates())

In [4]:
# Define a region extract function
def extract_region(name):
    for region in regions:
        if region in name:
            return region
    return None

df['ar_buyer_region'] = df['ar_buyer_standard'].astype(str).apply(extract_region)
df['cus_region'] = df['cus_name_standard'].astype(str).apply(extract_region)

In [11]:
# City match for ar_buyer and cus_name, also keep the N.A.
df_citymatch = df[
    (df['ar_buyer_region'] == df['cus_region']) |
    (df['ar_buyer_region'].isna() & df['cus_region'].isna())
]

In [14]:
df_citymatch.head(10)

,wind_code,sec_name,ar_by_buyer_name,ar_buyer_standard,ar_buyer_subid,cus_name,cus_name_standard,cus_name_subid,ar_buyer_region,cus_region
77,000004.SZ,国华网安,内蒙古公安厅,内蒙古公安厅,6,国药乐仁堂医药有限公司,国药乐仁堂医药有限公司,3,None,None
107,000004.SZ,国华网安,周文亮,周文亮,8,国药乐仁堂医药有限公司,国药乐仁堂医药有限公司,3,None,None
121,000004.SZ,国华网安,四川科伦医药贸易有限公司,四川科伦医药贸易有限公司,9,四川源亨众生医药有限公司,四川源亨众生医药有限公司,2,四川,四川
136,000004.SZ,国华网安,国药控股四川医药股份有限公司,国药控股四川医药股份有限公司,10,四川源亨众生医药有限公司,四川源亨众生医药有限公司,2,四川,四川
152,000004.SZ,国华网安,宁夏耀康医药有限公司,宁夏耀康医药有限公司,11,国药乐仁堂医药有限公司,国药乐仁堂医药有限公司,3,None,None
183,000004.SZ,国华网安,山东瑞康医药股份有限公司,山东瑞康医药股份有限公司,13,山东众智医药有限公司,山东众智医药有限公司,4,山东,山东
198,000004.SZ,国华网安,山东豪诺医药有限公司,山东豪诺医药有限公司,14,山东众智医药有限公司,山东众智医药有限公司,4,山东,山东
302,000004.SZ,国华网安,杜秀珍,杜秀珍,21,国药乐仁堂医药有限公司,国药乐仁堂医药有限公司,3,None,None
317,000004.SZ,国华网安,"杨秀兰,丁亮","杨秀兰,丁亮",22,国药乐仁堂医药有限公司,国药乐仁堂医药有限公司,3,None,None
351,000004.SZ,国华网安,河北龙海新药经营有限公司,河北龙海新药经营有限公司,24,河北启源医药有限公司,河北启源医药有限公司,7,河北,河北


### Step 2: Fuzzy match

In [15]:
df_fuzzymatch = df_citymatch.iloc[:,:]

# Fuzzy match function
def fuzzy_match(name1, name2):
    return fuzz.partial_ratio(name1, name2)

# Apply fuzzy matching to your columns
df_fuzzymatch['match_score'] = df_fuzzymatch.apply(lambda row: fuzzy_match(row['cus_name_standard'], row['ar_buyer_standard']), axis=1)

C:\Users\Shuning TIE\AppData\Local\Temp\ipykernel_19080\971068601.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fuzzymatch['match_score'] = df_fuzzymatch.apply(lambda row: fuzzy_match(row['cus_name_standard'], row['ar_buyer_standard']), axis=1)


In [20]:
df_fuzzymatch.head(10)

,wind_code,sec_name,ar_by_buyer_name,ar_buyer_standard,ar_buyer_subid,cus_name,cus_name_standard,cus_name_subid,ar_buyer_region,cus_region,match_score
77,000004.SZ,国华网安,内蒙古公安厅,内蒙古公安厅,6,国药乐仁堂医药有限公司,国药乐仁堂医药有限公司,3,None,None,18
107,000004.SZ,国华网安,周文亮,周文亮,8,国药乐仁堂医药有限公司,国药乐仁堂医药有限公司,3,None,None,0
121,000004.SZ,国华网安,四川科伦医药贸易有限公司,四川科伦医药贸易有限公司,9,四川源亨众生医药有限公司,四川源亨众生医药有限公司,2,四川,四川,67
136,000004.SZ,国华网安,国药控股四川医药股份有限公司,国药控股四川医药股份有限公司,10,四川源亨众生医药有限公司,四川源亨众生医药有限公司,2,四川,四川,73
152,000004.SZ,国华网安,宁夏耀康医药有限公司,宁夏耀康医药有限公司,11,国药乐仁堂医药有限公司,国药乐仁堂医药有限公司,3,None,None,60
183,000004.SZ,国华网安,山东瑞康医药股份有限公司,山东瑞康医药股份有限公司,13,山东众智医药有限公司,山东众智医药有限公司,4,山东,山东,60
198,000004.SZ,国华网安,山东豪诺医药有限公司,山东豪诺医药有限公司,14,山东众智医药有限公司,山东众智医药有限公司,4,山东,山东,80
302,000004.SZ,国华网安,杜秀珍,杜秀珍,21,国药乐仁堂医药有限公司,国药乐仁堂医药有限公司,3,None,None,0
317,000004.SZ,国华网安,"杨秀兰,丁亮","杨秀兰,丁亮",22,国药乐仁堂医药有限公司,国药乐仁堂医药有限公司,3,None,None,0
351,000004.SZ,国华网安,河北龙海新药经营有限公司,河北龙海新药经营有限公司,24,河北启源医药有限公司,河北启源医药有限公司,7,河北,河北,50


In [21]:
output_path1 = os.path.join(root_dir, "df_fuzzymatch.csv")
df_fuzzymatch.to_csv(output_path1, encoding = 'utf-8-sig')

#### (1) Choose 85 as a cutoff score

In [22]:
df_fuzzymatch85 = df_fuzzymatch[df_fuzzymatch['match_score'] >= 85]
df_fuzzymatch85.head(10)

,wind_code,sec_name,ar_by_buyer_name,ar_buyer_standard,ar_buyer_subid,cus_name,cus_name_standard,cus_name_subid,ar_buyer_region,cus_region,match_score
430,000004.SZ,国华网安,福建乐游网络科技有限公司,福建乐游网络科技有限公司,29,福建乐游网络科技有限公司,福建乐游网络科技有限公司,11,福建,福建,100
446,000004.SZ,国华网安,福建众邦药业有限公司,福建众邦药业有限公司,30,福建众邦药业有限公司,福建众邦药业有限公司,12,福建,福建,100
462,000004.SZ,国华网安,福建摩格网络科技有限公司,福建摩格网络科技有限公司,31,福建摩格网络科技有限公司,福建摩格网络科技有限公司,13,福建,福建,100
551,000006.SZ,深振业A,天虹商场股份有限公司,天虹商场股份有限公司,2,天虹商场股份有限公司,天虹商场股份有限公司,3,None,None,100
599,000006.SZ,深振业A,深圳市前海同创汇产业运营管理有限公司,深圳市前海同创汇产业运营管理有限公司,7,深圳市前海同创汇产业运营管理有限公司,深圳市前海同创汇产业运营管理有限公司,6,深圳,深圳,100
619,000006.SZ,深振业A,深圳市南山区建筑工务署,深圳市南山区建筑工务署,9,深圳市南山区建筑工务署,深圳市南山区建筑工务署,8,深圳,深圳,100
797,000007.SZ,全新好,MAPLE PACKING INDUSTRIAL (H.K.) LIMITED,MAPLE PACKING INDUSTRIAL (H.K.) LIMITED,5,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,1,None,None,89
814,000007.SZ,全新好,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,6,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,1,None,None,100
815,000007.SZ,全新好,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,6,WANDEINDUSTRIAL(HK)LIMITED,WANDEINDUSTRIAL(HK)LIMITED,2,None,None,85
900,000007.SZ,全新好,WANDE INDUSTRIAL (HK) LIMITED,WANDE INDUSTRIAL (HK) LIMITED,11,WANDEINDUSTRIAL(HK)LIMITED,WANDEINDUSTRIAL(HK)LIMITED,2,None,None,88


In [23]:
output_path2 = os.path.join(root_dir, "df_fuzzymatch85.csv")
df_fuzzymatch85.to_csv(output_path2, encoding = 'utf-8-sig')

#### (1) Choose 90 as a cutoff score

In [24]:
df_fuzzymatch90 = df_fuzzymatch[df_fuzzymatch['match_score'] >= 90]
df_fuzzymatch90.head(10)

,wind_code,sec_name,ar_by_buyer_name,ar_buyer_standard,ar_buyer_subid,cus_name,cus_name_standard,cus_name_subid,ar_buyer_region,cus_region,match_score
430,000004.SZ,国华网安,福建乐游网络科技有限公司,福建乐游网络科技有限公司,29,福建乐游网络科技有限公司,福建乐游网络科技有限公司,11,福建,福建,100
446,000004.SZ,国华网安,福建众邦药业有限公司,福建众邦药业有限公司,30,福建众邦药业有限公司,福建众邦药业有限公司,12,福建,福建,100
462,000004.SZ,国华网安,福建摩格网络科技有限公司,福建摩格网络科技有限公司,31,福建摩格网络科技有限公司,福建摩格网络科技有限公司,13,福建,福建,100
551,000006.SZ,深振业A,天虹商场股份有限公司,天虹商场股份有限公司,2,天虹商场股份有限公司,天虹商场股份有限公司,3,None,None,100
599,000006.SZ,深振业A,深圳市前海同创汇产业运营管理有限公司,深圳市前海同创汇产业运营管理有限公司,7,深圳市前海同创汇产业运营管理有限公司,深圳市前海同创汇产业运营管理有限公司,6,深圳,深圳,100
619,000006.SZ,深振业A,深圳市南山区建筑工务署,深圳市南山区建筑工务署,9,深圳市南山区建筑工务署,深圳市南山区建筑工务署,8,深圳,深圳,100
814,000007.SZ,全新好,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,6,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,1,None,None,100
917,000007.SZ,全新好,WANDEINDUSTRIAL(HK)LIMITED,WANDEINDUSTRIAL(HK)LIMITED,12,WANDEINDUSTRIAL(HK)LIMITED,WANDEINDUSTRIAL(HK)LIMITED,2,None,None,100
1484,000007.SZ,全新好,深圳市正和丰实业有限公司,深圳市正和丰实业有限公司,45,深圳市正和丰实业有限公司,深圳市正和丰实业有限公司,8,深圳,深圳,100
1523,000007.SZ,全新好,深圳粤新商业管理有限公司,深圳粤新商业管理有限公司,47,深圳粤新商业管理有限公司,深圳粤新商业管理有限公司,13,深圳,深圳,100


In [25]:
output_path3 = os.path.join(root_dir, "df_fuzzymatch90.csv")
df_fuzzymatch90.to_csv(output_path3, encoding = 'utf-8-sig')